In [ ]:
import os
import time
import joblib
import numpy as np
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# 設定路徑
WATCH_FOLDER = '/Users/hoyi/Desktop/HI'
RESULT_FILE = '/Users/hoyi/Desktop/new.txt'
MODEL_PATH = '/Users/hoyi/Downloads/eeg_model/random_forest_model.joblib'
SCALER_PATH = '/Users/hoyi/Downloads/eeg_model/standard_scaler.joblib'

# 載入模型和標準化器
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# 已處理過的檔案集合
processed_files = set()

class NewFileHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return
            
        if event.src_path.endswith('.txt') and event.src_path not in processed_files:
            print(f"偵測到新檔案: {event.src_path}")
            process_file(event.src_path)
            processed_files.add(event.src_path)

def process_file(file_path):
    try:
        # 讀取檔案內容
        with open(file_path, 'r') as f:
            content = f.read().strip()
            
        # 解析數值
        try:
            beta, alpha = map(float, content.split(','))
            features = np.array([[beta, alpha]])
            
            # 標準化特徵
            features_scaled = scaler.transform(features)
            
            # 預測 (直接得到 1, 2 或 3)
            prediction = model.predict(features_scaled)[0]
            
            # 只寫入預測數字到結果文件
            with open(RESULT_FILE, 'a') as f:
                f.write(f"{prediction}\n")
                
            print(f"處理完成: {file_path} -> 預測類別: {prediction}")
            
        except ValueError as e:
            print(f"檔案內容格式錯誤: {file_path} - {str(e)}")
            with open(RESULT_FILE, 'a') as f:
                f.write("格式錯誤\n")
                
    except Exception as e:
        print(f"處理檔案時發生錯誤: {file_path} - {str(e)}")

def main():
    # 初始化監控
    event_handler = NewFileHandler()
    observer = Observer()
    observer.schedule(event_handler, WATCH_FOLDER, recursive=False)
    observer.start()
    
    # 檢查已存在的檔案
    for filename in os.listdir(WATCH_FOLDER):
        if filename.endswith('.txt'):
            file_path = os.path.join(WATCH_FOLDER, filename)
            if file_path not in processed_files:
                process_file(file_path)
                processed_files.add(file_path)
    
    print(f"開始監控資料夾: {WATCH_FOLDER}")
    print(f"結果將寫入: {RESULT_FILE} (只包含預測數字 1, 2 或 3)")
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

if __name__ == "__main__":
    # 初始化結果文件 (如果不存在則創建)
    if not os.path.exists(RESULT_FILE):
        with open(RESULT_FILE, 'w') as f:
            pass  # 創建空文件
    
    main()

In [ ]:
import os
import time
import joblib
import numpy as np
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

WATCH_FOLDER = '/Users/hoyi/Desktop/HI'
RESULT_FILE = '/Users/hoyi/Desktop/cluster_result.txt'
MODEL_PATH = '/Users/hoyi/Downloads/eeg_model/random_forest_model.joblib'
SCALER_PATH = '/Users/hoyi/Downloads/eeg_model/standard_scaler.joblib'

model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

alpha_list = []
beta_list = []
alpha_avg = None
beta_avg = None
alpha_upper = None
beta_upper = None
alpha_lower = None
beta_lower = None
baseline_samples = 60
processed_files = set()
deleted_count = 0

class EEGFileHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.endswith('.txt'):
            return

        print(f"偵測到新檔案: {event.src_path}")
        process_file(event.src_path)

def process_file(filepath):
    global alpha_list, beta_list
    global alpha_avg, beta_avg, alpha_upper, beta_upper, alpha_lower, beta_lower, deleted_count
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()
        data = np.array([float(x.strip()) for x in lines])

        if len(data) < 28:
            print("資料不足")
            return

        alpha = np.mean(data[7:12])
        beta = np.mean(data[11:28])

        if len(alpha_list) < baseline_samples:
            alpha_list.append(alpha)
            beta_list.append(beta)
            print(f"累積基準值資料：{len(alpha_list)}/{baseline_samples}")
            if len(alpha_list) == baseline_samples:
                alpha_q1, alpha_q3 = np.percentile(alpha_list, [25, 75])
                beta_q1, beta_q3 = np.percentile(beta_list, [25, 75])

                alpha_upper = alpha_q3 + 1.5 * (alpha_q3 - alpha_q1)
                alpha_lower = alpha_q1 - 1.5 * (alpha_q3 - alpha_q1)
                beta_upper = beta_q3 + 1.5 * (beta_q3 - beta_q1)
                beta_lower = beta_q1 - 1.5 * (beta_q3 - beta_q1)

                # 同時為離群值者移除
                keep_indices = ~((np.array(alpha_list) < alpha_lower) | (np.array(alpha_list) > alpha_upper)) | \
                               ~((np.array(beta_list) < beta_lower) | (np.array(beta_list) > beta_upper))

                removed = baseline_samples - np.sum(keep_indices)
                alpha_list = list(np.array(alpha_list)[keep_indices])
                beta_list = list(np.array(beta_list)[keep_indices])
                deleted_count += removed

                alpha_avg = np.mean(alpha_list)
                beta_avg = np.mean(beta_list)

                print(f"已建立基準，刪除 {removed} 筆離群，alpha_avg={alpha_avg:.4f}, beta_avg={beta_avg:.4f}")
            return

        # 已建立基準，進行判斷
        if alpha_lower <= alpha <= alpha_upper and beta_lower <= beta <= beta_upper:
            alpha_norm = (alpha - alpha_avg) / alpha_avg
            beta_norm = (beta - beta_avg) / beta_avg
            print(f"標準化後 alpha: {alpha_norm:.4f}, beta: {beta_norm:.4f}")

            features = np.array([[beta_norm, alpha_norm]])
            features_scaled = scaler.transform(features)
            prediction = model.predict(features_scaled)[0]

            with open(RESULT_FILE, 'a') as f:
                f.write(f"{prediction}\n")

            print(f"分類結果: {prediction}")
        else:
            deleted_count += 1
            print(f"檔案 {os.path.basename(filepath)} 為離群值，已忽略（總刪除: {deleted_count}）")

    except Exception as e:
        print(f"處理錯誤: {filepath} - {str(e)}")

def main():
    if not os.path.exists(RESULT_FILE):
        with open(RESULT_FILE, 'w') as f:
            pass

    event_handler = EEGFileHandler()
    observer = Observer()
    observer.schedule(event_handler, WATCH_FOLDER, recursive=False)
    observer.start()

    print(f"監控資料夾: {WATCH_FOLDER}")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

if __name__ == '__main__':
    main()


監控資料夾: /Users/hoyi/Desktop/HI
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234.txt
累積基準值資料：1/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝.txt
累積基準值資料：2/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝2.txt
累積基準值資料：3/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝3.txt
累積基準值資料：4/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝4.txt
累積基準值資料：5/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝5.txt
累積基準值資料：6/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝6.txt
累積基準值資料：7/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝7.txt
累積基準值資料：8/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝8.txt
累積基準值資料：9/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝9.txt
累積基準值資料：10/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝10.txt
累積基準值資料：11/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝11.txt
累積基準值資料：12/60
偵測到新檔案: /Users/hoyi/Desktop/HI/FFT_Data_20250405160234拷貝12.txt
累積基準值資

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # 用於導出模型
import os

# 1. 加載數據
file_path = '/Users/hoyi/Downloads/eeg_features_labels.csv'
data = pd.read_csv(file_path)

# 2. 準備特徵和標籤
X = data[['beta_mean', 'alpha_mean']].values
y = data['label'].values

# 3. 特徵標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ... [中間的代碼保持不變，直到最後的模型訓練部分] ...

# 13. 訓練最終模型
final_model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
final_model.fit(X_scaled, y)

# 14. 導出模型和標準化器
output_folder = '/Users/hoyi/Downloads/eeg_model3'
os.makedirs(output_folder, exist_ok=True)  # 創建文件夾如果不存在

# 保存模型
model_path = os.path.join(output_folder, 'random_forest_model.joblib')
joblib.dump(final_model, model_path)

# 保存標準化器 (因為預測新數據時也需要相同的標準化)
scaler_path = os.path.join(output_folder, 'standard_scaler.joblib')
joblib.dump(scaler, scaler_path)

print(f'\n模型已成功導出至: {model_path}')
print(f'標準化器已成功導出至: {scaler_path}')

# 15. (可選) 保存類別名稱
class_names_path = os.path.join(output_folder, 'class_names.joblib')
joblib.dump(class_names, class_names_path)


模型已成功導出至: /Users/hoyi/Downloads/eeg_model3/random_forest_model.joblib
標準化器已成功導出至: /Users/hoyi/Downloads/eeg_model3/standard_scaler.joblib


NameError: name 'class_names' is not defined

In [2]:
import sklearn
print(sklearn.__version__)

1.4.2
